# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [19]:
# 1. Fazendo a requisição à API e obtendo os dados
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()
pvl_data = data_json['items']

# 2. Criando um DataFrame a partir dos dados
df = pd.DataFrame(pvl_data)

# 3. Examinando os primeiros registros do DataFrame
print(df.head())

# 4. Obtendo os três status mais frequentes e suas frequências
top_status = df['status'].value_counts().head(3)
print("Três status mais frequentes:")
print(top_status)

# 5. Construindo uma nova variável com o ano do status
df['ano_status'] = df['data_status'].str.slice(0, 4)

# 6. Examinando os primeiros registros do DataFrame após a adição da nova variável
print(df.head())

# 7. Obtendo a frequência de cada ano do campo construído
freq_ano_status = df['ano_status'].value_counts()
print("Frequência de cada ano do campo construído:")
print(freq_ano_status)


   id_pleito tipo_interessado          interessado  cod_ibge  uf  \
0       5290        Município    Casimiro de Abreu   3301306  RJ   
1      61009        Município       Rio das Flores   3304508  RJ   
2      66228        Município  São Pedro da Aldeia   3305208  RJ   
3       6363        Município            Rio Claro   3304409  RJ   
4       7907        Município             Mesquita   3302858  RJ   

                num_pvl                          status          num_processo  \
0                  None                        Deferido  17944.000078/2014-81   
1  PVL02.007214/2022-19  Arquivado por decurso de prazo  17944.103071/2022-20   
2  PVL02.003177/2023-42               Deferido (PVL-IF)                  None   
3                  None                    Regularizado  17944.000328/2008-34   
4                  None                        Deferido  17944.000675/2010-81   

         data_protocolo                tipo_operacao  \
0  2014-11-28T02:00:00Z  Operação contratual int

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [21]:
import pandas as pd
import requests

def consulta_api(uf, tipo_interessado):
    # Definindo o URL base
    base_url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

    # Construindo os parâmetros da consulta
    params = {'uf': uf, 'tipo_interessado': tipo_interessado}

    try:
        # Fazendo a requisição à API
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Lança uma exceção para códigos de resposta diferentes de 2xx (sucesso)

        # Convertendo os dados JSON para um DataFrame
        data_json = response.json()
        df = pd.DataFrame(data_json['items'])

        return df

    except requests.exceptions.RequestException as err:
        print(f"Erro na requisição à API: {err}")
        return None

# Exemplo de uso da função para Minas Gerais com tipo_interessado='Estado'
df_minas_gerais = consulta_api(uf='MG', tipo_interessado='Estado')

if df_minas_gerais is not None:
    print("Consulta para Minas Gerais com tipo_interessado='Estado':")
    print(df_minas_gerais.head())
else:
    print("Consulta falhou. Verifique os parâmetros e tente novamente.")

# Exemplo de uso da função para Bahia com tipo_interessado='Município'
df_bahia_municipio = consulta_api(uf='BA', tipo_interessado='Município')

if df_bahia_municipio is not None:
    print("Consulta para Bahia com tipo_interessado='Município':")
    print(df_bahia_municipio.head())
else:
    print("Consulta falhou. Verifique os parâmetros e tente novamente.")


Consulta para Minas Gerais com tipo_interessado='Estado':
   id_pleito tipo_interessado   interessado  cod_ibge  uf num_pvl  \
0       4926           Estado  Minas Gerais        31  MG    None   
1       7280           Estado  Minas Gerais        31  MG    None   
2       7435           Estado  Minas Gerais        31  MG    None   
3      10338           Estado  Minas Gerais        31  MG    None   
4       5828           Estado  Minas Gerais        31  MG    None   

                                              status          num_processo  \
0                                          Arquivado  10951.000367/1998-24   
1                                           Deferido  17944.000537/2010-01   
2  Encaminhado à PGFN com manifestação técnica fa...  17944.000576/2009-66   
3  Encaminhado à PGFN com manifestação técnica fa...  17944.001175/2009-23   
4  Encaminhado à PGFN com manifestação técnica fa...  17944.000208/2013-02   

         data_protocolo                                   

In [24]:
# Exemplo de uso da função para Bahia com tipo_interessado='Estado'
df_bahia_estado = consulta_api(uf='BA', tipo_interessado='Estado')

if df_bahia_estado is not None:
    # Salvando os dados em um arquivo CSV
    df_bahia_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)
    print("Dados da Bahia com tipo_interessado='Estado' salvos em solicitacoes_bahia_estado.csv")
else:
    print("Consulta falhou. Verifique os parâmetros e tente novamente.")


Dados da Bahia com tipo_interessado='Estado' salvos em solicitacoes_bahia_estado.csv
